In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bladder_Cancer"
cohort = "GSE145261"

# Input paths
in_trait_dir = "../../input/GEO/Bladder_Cancer"
in_cohort_dir = "../../input/GEO/Bladder_Cancer/GSE145261"

# Output paths
out_data_file = "../../output/preprocess/Bladder_Cancer/GSE145261.csv"
out_gene_data_file = "../../output/preprocess/Bladder_Cancer/gene_data/GSE145261.csv"
out_clinical_data_file = "../../output/preprocess/Bladder_Cancer/clinical_data/GSE145261.csv"
json_path = "../../output/preprocess/Bladder_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Urothelial-to-Neural Lineage Plasticity Drives Progression to Small Cell Bladder Cancer"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Small cell carcinoma (SCC) of the bladder displays a high propensity for distant metastasis and is associated with short survival. We report a comprehensive molecular analysis of 34 cases of SCC and 84 cases of conventional urothelial carcinoma (UC)"
Sample Characteristics Dictionary:
{0: ['subject age: 72 years', 'subject age: 76 years', 'subject age: 79 years', 'subject age: 60 years', 'subject age: 65 years', 'subject age: 41 years', 'subject age: 67 years', 'subject age: 71 years', 'subject age: 57 years', 'subject age: 34 years', 'subject age: 62 years', 'subject age: 90 years', 'subject age: 58 years'], 1: ['subject gender: male', 'subject gender: female'], 2: ['tissue: bladder'], 3: ['tissue type: small cell carinoma (SCC)']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on background information, this is a study on bladder cancer with molecular analysis,
# likely to contain gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait: Based on sample characteristics dict, tissue type is indicated in key 3
trait_row = 3
# For age: Age information is in key 0
age_row = 0
# For gender: Gender information is in key 1
gender_row = 1

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert bladder cancer type to binary (0=not SCC, 1=SCC)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if 'small cell' in value.lower() or 'scc' in value.lower():
        return 1  # SCC bladder cancer
    else:
        return 0  # Not SCC bladder cancer

def convert_age(value):
    """Convert age value to continuous numeric"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Extract numeric age value
    import re
    match = re.search(r'(\d+)', value)
    if match:
        return int(match.group(1))
    else:
        return None

def convert_gender(value):
    """Convert gender to binary (0=female, 1=male)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    value = value.lower()
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    else:
        return None

# 3. Save Metadata
# Trait data is available if trait_row is not None
is_trait_available = trait_row is not None

# Save initial filtering info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Use the library function to extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    print("Clinical data preview:")
    print(preview_df(clinical_df))
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical data preview:
{'GSM4310302': [1.0, 72.0, 1.0], 'GSM4310303': [1.0, 76.0, 1.0], 'GSM4310304': [1.0, 72.0, 1.0], 'GSM4310305': [1.0, 79.0, 1.0], 'GSM4310306': [1.0, 60.0, 1.0], 'GSM4310307': [1.0, 65.0, 1.0], 'GSM4310308': [1.0, 41.0, 1.0], 'GSM4310309': [1.0, 76.0, 0.0], 'GSM4310310': [1.0, 76.0, 0.0], 'GSM4310311': [1.0, 67.0, 1.0], 'GSM4310312': [1.0, 71.0, 1.0], 'GSM4310313': [1.0, 65.0, 1.0], 'GSM4310314': [1.0, 71.0, 1.0], 'GSM4310315': [1.0, 72.0, 1.0], 'GSM4310316': [1.0, 57.0, 1.0], 'GSM4310317': [1.0, 71.0, 1.0], 'GSM4310318': [1.0, 67.0, 1.0], 'GSM4310319': [1.0, 34.0, 1.0], 'GSM4310320': [1.0, 62.0, 1.0], 'GSM4310321': [1.0, 90.0, 0.0], 'GSM4310322': [1.0, 72.0, 1.0], 'GSM4310323': [1.0, 58.0, 1.0]}
Clinical data saved to ../../output/preprocess/Bladder_Cancer/clinical_data/GSE145261.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These are Illumina BeadArray identifiers (ILMN_*), not human gene symbols
# They need to be mapped to proper gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns in gene_annotation for mapping
# From the previous output:
# - 'ID' is the column with the same identifiers (ILMN_*) as in gene_expression data
# - 'Symbol' contains the gene symbols we need to map to

# 2. Create a gene mapping dataframe using get_gene_mapping function
# The function extracts and processes these two columns
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Symbol')

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# This function handles the many-to-many relationships between probes and genes
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Show a preview of the new gene expression data
print("Gene expression data after mapping (first 5 rows, 5 columns):")
preview_cols = min(5, len(gene_data.columns))
print(gene_data.iloc[:5, :preview_cols])
print(f"Shape after mapping: {gene_data.shape}")


Gene expression data after mapping (first 5 rows, 5 columns):
       GSM4310302  GSM4310303  GSM4310304  GSM4310305  GSM4310306
Gene                                                             
A1BG    18.653255   18.667788   18.862924   18.728951   18.664749
A1CF    27.961683   28.422059   27.961613   27.960573   27.959180
A26C3   27.959176   29.677706   28.149562   28.236884   27.964586
A2BP1   37.291573   38.591834   37.474717   38.672140   41.538859
A2LD1    9.361814    9.418669    9.316703    9.596790    9.376270
Shape after mapping: (21464, 22)


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data and link with genetic data
clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(True, cohort, json_path, True, True, is_trait_biased, unbiased_linked_data)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Bladder_Cancer/gene_data/GSE145261.csv


Quartiles for 'Bladder_Cancer':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Bladder_Cancer' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 62.75
  50% (Median): 71.0
  75%: 72.0
Min: 34.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 3 occurrences. This represents 13.64% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Data was determined to be unusable and was not saved
